In [0]:
# Setting Catalog and Schema
catalog_name = dbutils.widgets.get("catalog")
schema_name = dbutils.widgets.get("schema")
volume_name = "raw"

spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")

# Location of the raw data
location_raw = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/"


In [0]:
# GET file_parquet and folder_volume param's
file_parquet = dbutils.widgets.get("file_parquet")
folder_volume = dbutils.widgets.get("folder_volume")

# # TESTE
# file_parquet = "./output/ProductionProduct/.ProductionProduct_20250610_1548.000.parquet"
# folder_volume = "database_embulk"

# Setting prefix_table
prefix_table = 'raw_'

if folder_volume == 'database_meltano':
    prefix_table += 'meltano_'
elif folder_volume == 'database_embulk':
    prefix_table += ''
else:
    prefix_table += ''


table_name = (prefix_table + 
              (file_parquet.replace("./output/","")
              [:(file_parquet.replace("./output/","")
                 .find("/"))]
              .lower()))

table_name = table_name.replace("-","_")
print(table_name)

file_parquet = f'{location_raw}{folder_volume}/{file_parquet.replace("./output/","")}'

try:
    sql_command = (f'''
                   COPY INTO {table_name}
                   FROM '{file_parquet}'
                   FILEFORMAT = PARQUET
                   ''')
    spark.sql(sql_command)
except Exception as e:
    df = spark.read.option("header", True).parquet(file_parquet)
    df.write.format("delta").mode("overwrite").saveAsTable(f"{table_name}")


bronze_productionproduct
